In this tutorial we will learn to:
- Instantiate a DeepPrintExtractor
- Prepare a training dataset
- Train a DeepPrintExtractor

## Instantiate a DeepPrintExtractor

This package implements a number of variants of the DeepPrint architecture. The wrapper class for all these variants is called `DeepPrintExtractor`.
It has a `fit` method to train (and save) the model as well as an `extract` method to extract the DeepPrint features for fingerprint images. 

You can also try to implement your own models, but currently this is not directly supported by the package.

In [4]:
from flx.data.dataset import IdentifierSet, Identifier
from flx.extractor.fixed_length_extractor import get_DeepPrint_TexMinu, DeepPrintExtractor

# We will use the example dataset with 10 subjects and 10 impression per subject
training_ids: IdentifierSet = IdentifierSet([Identifier(i, j) for i in range(100,110) for j in range(8)])

# We choose a dimension of 512 for the fixed-length representation (TexMinu has two outputs num_dims)
extractor: DeepPrintExtractor = get_DeepPrint_TexMinu(num_training_subjects=training_ids.num_subjects, num_dims=256)

Created IdentifierSet with 10 subjects and a total of 80 samples.


In [2]:
training_ids

## Training the model

Instantiating the model was easy. To train it, first we will load the training data (see the [data tutorial](./dataset_tutorial.ipynb) for how to implement your own dataset).

Besides the fingerprint images, we also need a mapping from subjects to integer labels (for pytorch). For some variants we also need minutiae data. To see how a more complex dataset can be loaded, have a look at `flx/setup/datasets.py`.

Finally, we call the `fit` method, which trains the model and saves it to the specified path.

There is also the option to add a validation set, which will be used to evaluate the embeddings during training. This is useful to monitor the training progress and to avoid overfitting.
In this example we will not use a validation set for simplicity.

In [5]:
import os

import torch 

from flx.data.dataset import *
from flx.data.image_loader import SFingeLoader
from flx.data.minutia_map_loader import SFingeMinutiaMapLoader
from flx.data.label_index import LabelIndex
from flx.data.transformed_image_loader import TransformedImageLoader
from flx.image_processing.binarization import LazilyAllocatedBinarizer
from flx.data.image_helpers import pad_and_resize_to_deepprint_input_size

# NOTE: If this does not work, enter the absolute paths manually here! 
DATASET_DIR: str = os.path.abspath("fvc_dataset/fvc2002_DB3_B capacitive/converted")
MODEL_OUTDIR: str = os.path.abspath("fvc2002_model")

# We will use the SFingeLoader to load the images from the dataset
image_loader = TransformedImageLoader(
        images=SFingeLoader(DATASET_DIR),
        poses=None,
        transforms=[
            LazilyAllocatedBinarizer(5.0),
            pad_and_resize_to_deepprint_input_size,
        ],
    )
print(training_ids.num_subjects)

image_dataset = Dataset(image_loader, training_ids)
print(image_dataset)
# For pytorch, we need to map the subjects to integer labels from [0 ... num_subjects-1]
label_dataset = Dataset(LabelIndex(training_ids), training_ids)

minutia_maps_dataset = Dataset(SFingeMinutiaMapLoader(DATASET_DIR), training_ids)

extractor.fit(
    fingerprints=image_dataset,
    minutia_maps=minutia_maps_dataset,
    labels=label_dataset,
    validation_fingerprints=None,
    validation_benchmark=None,
    num_epochs=250,
    out_dir=MODEL_OUTDIR
)

Created IdentifierSet with 10 subjects and a total of 80 samples.
10
Dataset with 10 subjects and a total of 80 samples.
Created IdentifierSet with 10 subjects and a total of 80 samples.
Using device cpu
Loaded existing model from /home/rs/21CS91R01/research/fixed-length-fingerprint-extractors/notebooks/fvc2002_model/model.pyt


 --- Starting Epoch 51 of 250 ---

Training:


100%|██████████| 5/5 [08:26<00:00, 101.33s/it]


Average Loss: 10.805987285971643
Multiclass accuracy: 0.5
TrainingLogEntry(
    epoch=51,
    training_loss=10.805987285971643,
    loss_statistics={'texture_loss': {'crossent_loss_sum': 0.10373715162277222, 'center_loss_sum': 0.003941927850246429}, 'minutia_loss': {'crossent_loss_sum': 0.1282074436545372, 'center_loss_sum': 0.0037042201962321996}, 'minutia_map_loss': 0.435783462189138},
    training_accuracy=0.5,
    validation_equal_error_rate=0.5,
}


 --- Starting Epoch 52 of 250 ---

Training:


100%|██████████| 5/5 [09:11<00:00, 110.22s/it]


Average Loss: 10.687256256341934
Multiclass accuracy: 0.3875000476837158
TrainingLogEntry(
    epoch=52,
    training_loss=10.687256256341934,
    loss_statistics={'texture_loss': {'crossent_loss_sum': 0.054192842543125154, 'center_loss_sum': 0.0017288472736254334}, 'minutia_loss': {'crossent_loss_sum': 0.06615822613239289, 'center_loss_sum': 0.001697778911329806}, 'minutia_map_loss': 0.2101990645006299},
    training_accuracy=0.3875000476837158,
    validation_equal_error_rate=0.3875000476837158,
}


 --- Starting Epoch 53 of 250 ---

Training:


...

Average Loss: 2.5405250219628215
Multiclass accuracy: 0.925000011920929
TrainingLogEntry(
    epoch=248,
    training_loss=2.5405250219628215,
    loss_statistics={'texture_loss': {'crossent_loss_sum': 0.0001073888126723092, 'center_loss_sum': 8.119357105419792e-06}, 'minutia_loss': {'crossent_loss_sum': 0.0005533355504575402, 'center_loss_sum': 1.6447501445207934e-05}, 'minutia_map_loss': 0.00011664217031052844},
    training_accuracy=0.925000011920929,
    validation_equal_error_rate=0.925000011920929,
}


 --- Starting Epoch 249 of 250 ---

Training:


100%|██████████| 5/5 [06:40<00:00, 80.15s/it]


Average Loss: 2.363038478754461
Multiclass accuracy: 0.9250000715255737
TrainingLogEntry(
    epoch=249,
    training_loss=2.363038478754461,
    loss_statistics={'texture_loss': {'crossent_loss_sum': 7.806306508318263e-05, 'center_loss_sum': 1.5250181816705507e-05}, 'minutia_loss': {'crossent_loss_sum': 0.0005191720310767093, 'center_loss_sum': 1.6194583389477515e-05}, 'minutia_map_loss': 0.00011348047025791887},
    training_accuracy=0.9250000715255737,
    validation_equal_error_rate=0.9250000715255737,
}


 --- Starting Epoch 250 of 250 ---

Training:


100%|██████████| 5/5 [01:09<00:00, 13.96s/it]


Average Loss: 2.3148404164612293
Multiclass accuracy: 0.9249999523162842
TrainingLogEntry(
    epoch=250,
    training_loss=2.3148404164612293,
    loss_statistics={'texture_loss': {'crossent_loss_sum': 6.748555321246386e-05, 'center_loss_sum': 1.115081668831408e-05}, 'minutia_loss': {'crossent_loss_sum': 0.000517459861934185, 'center_loss_sum': 1.503275753930211e-05}, 'minutia_map_loss': 0.00011225865916348994},
    training_accuracy=0.9249999523162842,
    validation_equal_error_rate=0.9249999523162842,
}
